In [1]:
import news_collectordb

/home/sirjanhansda/miniconda3/envs/fuckmylife/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/sirjanhansda/miniconda3/envs/fuckmylife/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/sirjanhansda/miniconda3/envs/fuckmylife/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in

In [2]:
rmp = news_collectordb.main()

/home/sirjanhansda/temp/web_scrapperdb/news_collectordb.py:208: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame([row])], ignore_index=True)


In [3]:
rmp

,Earnings,Revenue,Margins,Dividend,EBITDA,Debt,Sentiment,Trendlyne,ticker,rating,verdict
0,0.574132,0.670806,0.524539,0.654763,0.689867,0.469659,0.405220,0.125,ADANIPORTS.NS,0.0,0.0
1,-0.396707,-0.343971,-0.285471,0.073063,-0.186178,-0.521244,-0.306572,0.125,ASIANPAINT.NS,0.0,0.0
2,0.408274,0.385765,0.221008,0.453650,0.395080,0.326000,0.197438,0.125,AXISBANK.NS,0.0,0.0
3,0.529573,0.511497,0.286836,0.535839,0.521397,0.204052,0.428682,0.125,BAJFINANCE.NS,0.0,0.0
4,0.548803,0.566155,0.390253,0.526871,0.542959,0.299601,0.373455,0.125,BAJFINSERV.NS,0.0,0.0
5,0.437524,0.429265,0.301773,0.509499,0.470819,0.148699,0.400822,0.125,BHARTIARTL.NS,0.0,0.0
6,0.081370,0.261293,-0.025291,0.303667,0.055362,-0.190446,0.035216,0.125,BPCL.NS,0.0,0.0
7,0.690983,0.685224,0.510922,0.713371,0.621476,0.470883,0.546990,0.125,BRITANNIA.NS,0.0,0.0
8,0.428020,0.220443,0.295610,0.505507,0.387104,-0.037622,0.168131,0.125,CIPLA.NS,0.0,0.0
9,-0.442803,-0.371084,-0.314001,-0.204456,-0.301353,-0.532543,-0.335116,0.125,COALINDIA.NS,0.0,0.0


In [10]:
ticker_aliases = {
    'ADANIPORTS.NS': ['Adani Ports'],
    'APOLLOHOSP.NS': ['Apollo Hospitals'],
    'ASIANPAINT.NS': ['Asian Paints'],
    'AXISBANK.NS': ['Axis Bank'],
    'BAJAJ-AUTO.NS': ['Bajaj Auto'],
    'BAJFINANCE.NS': ['Bajaj Finance'],
    'BAJAJFINSV.NS': ['Bajaj Finserv'],
    'BHARTIARTL.NS': ['Bharti Airtel'],
    'BPCL.NS': ['Bharat Petroleum', 'BPCL'],
    'BRITANNIA.NS': ['Britannia'],
    'CIPLA.NS': ['Cipla'],
    'COALINDIA.NS': ['Coal India'],
    'DIVISLAB.NS': ['Divi\'s Labs'],
    'DRREDDY.NS': ['Dr. Reddy\'s', 'Dr. Reddy\'s Labs'],
    'EICHERMOT.NS': ['Eicher Motors'],
    'GRASIM.NS': ['Grasim Industries', 'Grasim'],
    'HCLTECH.NS': ['HCL Technologies', 'HCL Tech'],
    'HDFC.NS': ['HDFC'],
    'HDFCBANK.NS': ['HDFC Bank'],
    'HDFCLIFE.NS': ['HDFC Life'],
    'HEROMOTOCO.NS': ['Hero MotoCorp'],
    'HINDALCO.NS': ['Hindalco'],
    'HINDUNILVR.NS': ['Hindustan Unilever'],
    'ICICIBANK.NS': ['ICICI Bank'],
    'INDUSINDBK.NS': ['IndusInd Bank'],
    'INFY.NS': ['Infosys'],
    'ITC.NS': ['ITC'],
    'JSWSTEEL.NS': ['JSW Steel'],
    'KOTAKBANK.NS': ['Kotak Mahindra Bank', 'Kotak Bank'],
    'LT.NS': ['Larsen & Toubro', 'L&T'],
    'MARUTI.NS': ['Maruti Suzuki'],
    'M&M.NS': ['Mahindra & Mahindra'],
    'NESTLEIND.NS': ['Nestle India'],
    'NTPC.NS': ['NTPC'],
    'ONGC.NS': ['Oil & Natural Gas Corporation', 'ONGC'],
    'POWERGRID.NS': ['Power Grid'],
    'RELIANCE.NS': ['Reliance Industries', 'Reliance'],
    'SBILIFE.NS': ['SBI Life'],
    'SBIN.NS': ['State Bank of India', 'SBI'],
    'SHREECEM.NS': ['Shree Cement'],
    'SUNPHARMA.NS': ['Sun Pharmaceutical'],
    'TATACONSUM.NS': ['Tata Consumer Products', 'Tata Consumer'],
    'TATAMOTORS.NS': ['Tata Motors'],
    'TATASTEEL.NS': ['Tata Steel'],
    'TCS.NS': ['Tata Consultancy Services', 'TCS'],
    'TECHM.NS': ['Tech Mahindra'],
    'TITAN.NS': ['Titan Company'],
    'ULTRACEMCO.NS': ['UltraTech Cement'],
    'UPL.NS': ['UPL'],
    'WIPRO.NS': ['Wipro']
}


In [11]:
len(ticker_aliases.keys())

50

In [1]:
import news_collectordb as ncd

/home/club/miniconda3/envs/UMCNext/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/club/miniconda3/envs/UMCNext/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [2]:
ret = ncd.main()

{'ADANIPORTS.NS': {'Earnings': 0.4943845070177509, 'Revenue': 0.5260651335120201, 'Margins': 0.33810545061085673, 'Dividend': 0.5107021801173687, 'EBITDA': 0.4999933401122689, 'Debt': 0.28320479263430054, 'Sentiment': 0.5255888834081847, 'Trendlyne': 0.5698141130534086, 'Stock Symbol': 0.5757946775240057, 'Rating': 0.5010286746798335, 'Verdict': 0.5044068115098136, 'Quarterly Metrics': 0.6566959896257946, 'Stakeholder Perception': 0.3059192915116587, 'Strategic Innovation': 0.5910572749480867, 'Equity Dynamics': 0.5523530174242822, 'Technological Proliferation': 0.38845058548741224, 'Executive Foresight': 0.5152019076049328, 'Valuation Analysis': 0.5172505475142423, 'Operational Synergy': 0.6025348064445314, 'Technological Advancements': 0.636776523079191, 'Economic Impact': 0.48649829626083374, 'Transformational Potential': 0.6156274403555918, 'Strategic Governance': 0.5505158103801109, 'Compliance Framework': 0.44530528879934744, 'Operational Trade-offs': 0.11366514829879111, 'Margin

In [3]:
import pandas as pd

In [5]:
ret.keys()

dict_keys(['ADANIPORTS.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BHARTIARTL.NS', 'BPCL.NS', 'BRITANNIA.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DIVISLAB.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'INDUSINDBK.NS', 'INFY.NS', 'ITC.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'LT.NS', 'MARUTI.NS', 'M&M.NS', 'NESTLEIND.NS', 'NTPC.NS', 'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBILIFE.NS', 'SBIN.NS', 'SHREECEM.NS', 'SUNPHARMA.NS', 'TATACONSUM.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'TCS.NS', 'TECHM.NS', 'TITAN.NS', 'ULTRACEMCO.NS', 'UPL.NS', 'WIPRO.NS'])

In [9]:
aspects = ret['BPCL.NS'].keys()

In [10]:
aspects

dict_keys(['Earnings', 'Revenue', 'Margins', 'Dividend', 'EBITDA', 'Debt', 'Sentiment', 'Trendlyne', 'Stock Symbol', 'Rating', 'Verdict', 'Quarterly Metrics', 'Stakeholder Perception', 'Strategic Innovation', 'Equity Dynamics', 'Technological Proliferation', 'Executive Foresight', 'Valuation Analysis', 'Operational Synergy', 'Technological Advancements', 'Economic Impact', 'Transformational Potential', 'Strategic Governance', 'Compliance Framework', 'Operational Trade-offs', 'Margin Compression', 'Sustainable Advantage', 'Risk Mitigation', 'Adaptive Capacity', 'Capital Deployment', 'Expenditure Management', 'Growth Trajectory', 'Economic Prudence', 'Stakeholder Assurance', 'Operational Prioritization', 'Organizational Redesign', 'Technological Disruption', 'Structural Evolution', 'Workforce Recomposition', 'Strategic Pivot', 'Value Realization', 'Market Fluctuations', 'Technological Legitimacy', 'Institutional Sentiment', 'Innovative Potential', 'Volatility Management', 'Strategic Inve

In [12]:
new_dict = dict()
cols = list(aspects) + ['ticker']
for col in cols:
    if col == 'ticker':
        new_dict[col] = list(ret.keys())
    else:
        new_dict[col] = list(ret[ticker][col] for ticker in ret.keys())


In [13]:
pd.DataFrame(new_dict)

,Earnings,Revenue,Margins,Dividend,EBITDA,Debt,Sentiment,Trendlyne,Stock Symbol,Rating,...,Market Dominance,Resource Efficiency,Strategic Alignment,Debt Restructuring,Cost Optimization,Fiscal Transparency,Structural Vulnerability,Sustainability Strategy,Strategic Transparency,ticker
0,0.494385,0.526065,0.338105,0.510702,0.499993,0.283205,0.525589,0.569814,0.575795,0.501029,...,0.273168,0.564063,0.584581,0.362165,0.511845,0.554651,-0.316718,0.545011,0.526371,ADANIPORTS.NS
1,0.754787,0.756658,0.680362,0.709297,0.759259,0.516974,0.701649,0.693635,0.699990,0.653662,...,0.710963,0.716308,0.725585,0.612555,0.733483,0.673557,-0.203191,0.719515,0.698705,APOLLOHOSP.NS
2,-0.494166,-0.419712,-0.468254,-0.309077,-0.363647,-0.497203,-0.818239,-0.147799,-0.336609,-0.213883,...,-0.393428,-0.109787,0.046540,-0.378958,-0.180933,-0.099176,-0.596221,0.021833,-0.037069,ASIANPAINT.NS
3,0.602679,0.518134,0.328755,0.530359,0.463490,0.229215,0.353485,0.529422,0.636623,0.525338,...,0.438943,0.626590,0.592245,0.477379,0.617464,0.629470,-0.486428,0.586232,0.580910,AXISBANK.NS
4,0.301785,0.362814,0.285041,0.316887,0.378003,0.068461,-0.011378,0.357318,0.231870,0.363194,...,0.440524,0.483383,0.536984,0.086985,0.399977,0.467616,-0.539429,0.480614,0.478207,BAJAJ-AUTO.NS
5,0.322325,0.407156,0.185373,0.454213,0.305887,-0.167935,0.304311,0.396952,0.267643,0.378361,...,0.368178,0.423768,0.485004,0.158604,0.463925,0.472681,-0.551361,0.481199,0.423028,BAJFINANCE.NS
6,0.529305,0.494560,0.354685,0.483931,0.490922,0.015231,0.357276,0.508780,0.503906,0.454008,...,0.520790,0.551141,0.569900,0.306697,0.518093,0.522927,-0.395763,0.515390,0.559319,BAJAJFINSV.NS
7,0.468764,0.456750,0.338916,0.527841,0.448351,0.173500,0.546222,0.470815,0.507508,0.472381,...,0.487258,0.539445,0.527759,0.338682,0.544068,0.507217,-0.357262,0.504104,0.529210,BHARTIARTL.NS
8,0.175088,0.261727,0.017002,0.378091,0.122129,-0.205294,-0.025017,0.359535,0.223045,0.325013,...,0.240143,0.381509,0.483884,0.049292,0.352554,0.456101,-0.545533,0.446432,0.424522,BPCL.NS
9,-0.183614,0.115372,-0.210957,0.184437,-0.153262,-0.375296,-0.315189,0.258443,0.015032,0.217876,...,0.223989,0.241389,0.407752,-0.378769,0.262508,0.300025,-0.590994,0.251505,0.348500,BRITANNIA.NS


In [ ]:
aspects = self.df2['BPCL.NS'].keys()

        new_dict = dict()
        cols = list(aspects) + ['ticker']
        for col in cols:
            if col == 'ticker':
                new_dict[col] = list(self.df2.keys())
            else:
                new_dict[col] = list(self.df2[ticker][col] for ticker in self.df2.keys())

        self.df2 = pd.DataFrame(new_dict)